In [1]:
## Python port from R codes for the article: Khoury, D.S., Cromer, D., Reynaldi, A. et al. Neutralizing antibody levels are highly predictive of immune protection from symptomatic SARS-CoV-2 infection. Nat Med 27, 1205–1211 (2021). https://doi.org/10.1038/s41591-021-01377-8

import pandas as pd
import numpy as np
import scipy
import scipy.stats
import scipy.stats as stats
from scipy import optimize
from scipy.stats import norm
from pprint import pprint

In [2]:
### Import Tables Required for Estimating Mean and SD for each study
IndividualNeutData_NormalisedbyConv = pd.read_csv("IndividualNeutData_NormalisedbyConv.csv")
IndividualNeutData_RawVaccineAndConv = pd.read_csv("IndividualNeutData_RawVaccineAndConv.csv")
EfficacyTable = pd.read_csv("EfficacyMeanTable_20210226.csv")

In [3]:
####Test for difference in variance between vaccine groups
data = IndividualNeutData_RawVaccineAndConv[IndividualNeutData_RawVaccineAndConv['Group'] == 'Vaccine'].groupby(['Study'])
groups = []
for group_name, group_df in data:
    groups.append(data.get_group(group_name)['TitreLog'].to_numpy())

pprint(scipy.stats.fligner(*groups))

FlignerResult(statistic=14.103646569626164, pvalue=0.04936830881752221)


In [4]:
###################################################################################################
#########Fitting Normal Distribution to Neut data with 

###Creating Table of all the estimated Means and SD after censoring for each study
#These will be used in model fitting and to estimate SEM

#negative log likelihood of normal distibrution model with censoring

ListOfEstimatedMeanSDafterCensoring = IndividualNeutData_RawVaccineAndConv.drop_duplicates(subset=['Study', 'Group'])[['Study', 'Group']]
ListOfEstimatedMeanSDafterCensoring['SD'] = np.NaN
ListOfEstimatedMeanSDafterCensoring['EstimateMean'] = np.NaN
ListOfEstimatedMeanSDafterCensoring['NumberIndividuals'] = np.NaN

for index, row in ListOfEstimatedMeanSDafterCensoring.iterrows():
    tempdata = IndividualNeutData_RawVaccineAndConv[
        (IndividualNeutData_RawVaccineAndConv['Study'] == row['Study']) &
        (IndividualNeutData_RawVaccineAndConv['Group'] == row['Group'])
    ]['TitreLog'].to_numpy()

    fitmdltemp = norm.fit(tempdata)

    ListOfEstimatedMeanSDafterCensoring.at[index, 'SD'] = fitmdltemp[1]
    ListOfEstimatedMeanSDafterCensoring.at[index, 'EstimateMean'] = fitmdltemp[0]
    ListOfEstimatedMeanSDafterCensoring.at[index, 'NumberIndividuals'] = len(tempdata)

ListOfEstimatedMeanSDafterCensoring.reset_index(drop=True, inplace=True)
ListOfEstimatedMeanSDafterCensoring

,Study,Group,SD,EstimateMean,NumberIndividuals
0,Pfizer,Vaccine,0.379668,2.348386,24.0
1,Pfizer,Conv,0.389783,2.029885,38.0
2,Moderna,Vaccine,0.206426,2.836753,14.0
3,Moderna,Conv,0.334182,2.201868,3.0
4,Astra,Vaccine,0.244085,1.412198,9.0
5,Astra,Conv,0.236166,1.743646,5.0
6,CoronaVac,Vaccine,0.492288,1.453241,41.0
7,CoronaVac,Conv,0.583655,2.150286,109.0
8,Novavac,Vaccine,0.523904,3.587500,20.0
9,Novavac,Conv,0.626048,2.986727,35.0


In [5]:
#### Next we wish to pool all the studies together (and still consider censoring) 
# and determine a pooled SD
PooledSDModelFit = norm.fit(IndividualNeutData_NormalisedbyConv['CentredRatio'].to_numpy())
PooledSD = PooledSDModelFit[1]
pprint(PooledSDModelFit)

(-0.01582117201364084, 0.43912716705545857)


In [6]:
### For the convalescence study in Melb - create duplicate row called 
#Conv to match Vaccine because for this group the ratio neut between 
# conv and vaccine is 1
temprow = ListOfEstimatedMeanSDafterCensoring[ListOfEstimatedMeanSDafterCensoring['Study'] == 'Convalescence'].copy()
temprow['Group'] = 'Conv'
ListOfEstimatedMeanSDafterCensoring = ListOfEstimatedMeanSDafterCensoring.append([temprow], ignore_index=True)
ListOfEstimatedMeanSDafterCensoring

,Study,Group,SD,EstimateMean,NumberIndividuals
0,Pfizer,Vaccine,0.379668,2.348386,24.0
1,Pfizer,Conv,0.389783,2.029885,38.0
2,Moderna,Vaccine,0.206426,2.836753,14.0
3,Moderna,Conv,0.334182,2.201868,3.0
4,Astra,Vaccine,0.244085,1.412198,9.0
5,Astra,Conv,0.236166,1.743646,5.0
6,CoronaVac,Vaccine,0.492288,1.453241,41.0
7,CoronaVac,Conv,0.583655,2.150286,109.0
8,Novavac,Vaccine,0.523904,3.587500,20.0
9,Novavac,Conv,0.626048,2.986727,35.0


In [7]:
###Creating a Table where we will calculate ratio of neut vaccine vs conv
# for each study using these means from censored normal distribution fit
RowIndex = ListOfEstimatedMeanSDafterCensoring[ListOfEstimatedMeanSDafterCensoring['Group'] != 'Conv'][["Study","Group","SD"]]
CalculatingRatioandSEMafterCensoring = RowIndex.copy()
CalculatingRatioandSEMafterCensoring['NeutRatio_cens'] = np.NaN
CalculatingRatioandSEMafterCensoring['NeutConv_cens'] = np.NaN
CalculatingRatioandSEMafterCensoring['NeutVaccine_cens'] = np.NaN
CalculatingRatioandSEMafterCensoring['NumberIndividuals_Conv'] = np.NaN
CalculatingRatioandSEMafterCensoring['NumberIndividuals_Vaccine'] = np.NaN
CalculatingRatioandSEMafterCensoring['SD_Conv'] = np.NaN
CalculatingRatioandSEMafterCensoring['SD_Vaccine'] = np.NaN
CalculatingRatioandSEMafterCensoring['SEM'] = np.NaN

RowIndex.reset_index(drop=True, inplace=True)
CalculatingRatioandSEMafterCensoring.reset_index(drop=True, inplace=True)

for index, row in RowIndex.iterrows():

    #Calculate Ratio Vaccine to Conv Neut
    CalculatingRatioandSEMafterCensoring.at[index, 'NeutVaccine_cens'] = ListOfEstimatedMeanSDafterCensoring[(ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) & (ListOfEstimatedMeanSDafterCensoring['Group'] == row['Group'])]['EstimateMean']
    
    CalculatingRatioandSEMafterCensoring.at[index, 'NeutConv_cens'] = ListOfEstimatedMeanSDafterCensoring[(ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) & (ListOfEstimatedMeanSDafterCensoring['Group'] == "Conv")]['EstimateMean']

    #Number of Individuals used in estimate of Conv SD
    CalculatingRatioandSEMafterCensoring.at[index, 'NumberIndividuals_Conv'] = ListOfEstimatedMeanSDafterCensoring[(ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) & (ListOfEstimatedMeanSDafterCensoring['Group'] == 'Conv')]['NumberIndividuals']

    #Number of Individuals used in estimate of Vaccine SD
    CalculatingRatioandSEMafterCensoring.at[index, 'NumberIndividuals_Vaccine'] = ListOfEstimatedMeanSDafterCensoring[
      (ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) &
      (ListOfEstimatedMeanSDafterCensoring['Group'] == row['Group'])
    ]['NumberIndividuals']

    CalculatingRatioandSEMafterCensoring.at[index, 'SD_Vaccine'] = ListOfEstimatedMeanSDafterCensoring[
      (ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) &
      (ListOfEstimatedMeanSDafterCensoring['Group'] == row['Group'])
    ]['SD']

    CalculatingRatioandSEMafterCensoring.at[index, 'SD_Conv'] = ListOfEstimatedMeanSDafterCensoring[
      (ListOfEstimatedMeanSDafterCensoring['Study'] == row['Study']) &
      (ListOfEstimatedMeanSDafterCensoring['Group'] == 'Conv')
    ]['SD']     

CalculatingRatioandSEMafterCensoring['NeutRatio_cens'] = CalculatingRatioandSEMafterCensoring['NeutVaccine_cens'] - CalculatingRatioandSEMafterCensoring['NeutConv_cens']
CalculatingRatioandSEMafterCensoring['SEM'] = np.sqrt( ((CalculatingRatioandSEMafterCensoring['SD_Vaccine']**2)/CalculatingRatioandSEMafterCensoring['NumberIndividuals_Vaccine']) + ((CalculatingRatioandSEMafterCensoring['SD_Conv']**2)/CalculatingRatioandSEMafterCensoring['NumberIndividuals_Conv'])  ) 
CalculatingRatioandSEMafterCensoring


,Study,Group,SD,NeutRatio_cens,NeutConv_cens,NeutVaccine_cens,NumberIndividuals_Conv,NumberIndividuals_Vaccine,SD_Conv,SD_Vaccine,SEM
0,Pfizer,Vaccine,0.379668,0.318501,2.029885,2.348386,38.0,24.0,0.389783,0.379668,0.100022
1,Moderna,Vaccine,0.206426,0.634885,2.201868,2.836753,3.0,14.0,0.334182,0.206426,0.200673
2,Astra,Vaccine,0.244085,-0.331447,1.743646,1.412198,5.0,9.0,0.236166,0.244085,0.133322
3,CoronaVac,Vaccine,0.492288,-0.697045,2.150286,1.453241,109.0,41.0,0.583655,0.492288,0.095059
4,Novavac,Vaccine,0.523904,0.600773,2.986727,3.587500,35.0,20.0,0.626048,0.523904,0.157867
5,JJ,Vaccine,0.435201,-0.419335,2.809326,2.389991,32.0,54.0,0.593364,0.435201,0.120457
6,Sputnik,Vaccine,0.357453,0.152003,1.540366,1.692369,47.0,20.0,0.653066,0.357453,0.124350
7,Convalescence,Vaccine,0.474093,0.000000,1.687714,1.687714,64.0,64.0,0.474093,0.474093,0.083809


In [8]:
##### Building Summary Table with all SD and Means and Efficacy's
# This table will be used for most model fitting and figure creation

SummaryTable_Efficacy_NeutRatio_SD_SEM = EfficacyTable.merge(CalculatingRatioandSEMafterCensoring, how='left', left_on='Study', right_on='Study')
SummaryTable_Efficacy_NeutRatio_SD_SEM['PooledSD'] = PooledSD
SummaryTable_Efficacy_NeutRatio_SD_SEM['MelbSD'] = CalculatingRatioandSEMafterCensoring[CalculatingRatioandSEMafterCensoring['Study'] == "Convalescence"]['SD'].iloc[0]
### The Neutralisation ratio of vaccine to convalescence using reported neut titres.
SummaryTable_Efficacy_NeutRatio_SD_SEM['NeutRatio_Reported'] = np.log10(SummaryTable_Efficacy_NeutRatio_SD_SEM['NeutMean']/SummaryTable_Efficacy_NeutRatio_SD_SEM['NeutConv'])

SummaryTable_Efficacy_NeutRatio_SD_SEM.to_csv('python_SummaryTable_Efficacy_NeutRatio_SD_SEM.csv')
SummaryTable_Efficacy_NeutRatio_SD_SEM



,Study,Age,Efficacy,NumCont,NumVac,InfCont,InfVac,DayNeut,NeutMean,NeutConv,...,NeutConv_cens,NeutVaccine_cens,NumberIndividuals_Conv,NumberIndividuals_Vaccine,SD_Conv,SD_Vaccine,SEM,PooledSD,MelbSD,NeutRatio_Reported
0,Pfizer,all,0.9503,18325,18198,162,8,28,223.0000,94.0000,...,2.029885,2.348386,38.0,24.0,0.389783,0.379668,0.100022,0.439127,0.474093,0.375177
1,Moderna,all,0.9408,14073,14134,185,11,43,654.3000,158.3000,...,2.201868,2.836753,3.0,14.0,0.334182,0.206426,0.200673,0.439127,0.474093,0.616296
2,Sputnik,all,0.9155,4902,14964,62,16,42,49.2500,35.0000,...,1.540366,1.692369,47.0,20.0,0.653066,0.357453,0.124350,0.439127,0.474093,0.148338
3,Novavac,all,0.9566,7495,7505,23,1,35,3906.0000,983.0000,...,2.986727,3.587500,35.0,20.0,0.626048,0.523904,0.157867,0.439127,0.474093,0.599179
4,JJ,all,0.6734,18603,18604,300,98,29,245.5000,522.0000,...,2.809326,2.389991,32.0,54.0,0.593364,0.435201,0.120457,0.439127,0.474093,-0.327619
5,Astra,all,0.6184,4455,4440,71,27,42,32.0000,59.0000,...,1.743646,1.412198,5.0,9.0,0.236166,0.244085,0.133322,0.439127,0.474093,-0.265702
6,CoronaVac,all,0.5040,6416,6584,167,85,56,27.6000,163.7000,...,2.150286,1.453241,109.0,41.0,0.583655,0.492288,0.095059,0.439127,0.474093,-0.773140
7,Convalescence,all,0.8900,11276,1177,197,2,28,55.7943,55.7943,...,1.687714,1.687714,64.0,64.0,0.474093,0.474093,0.083809,0.439127,0.474093,0.000000
